# Spark 运行环境

```python
# 创建 SparkConf 和 SparkSession
conf=SparkConf()\
        .setMaster('local[*]')\
        .setAppName("WordCount")\
        .setExecutorEnv("spark.executor.memory","2g")\
        .setExecutorEnv("spark.driver.memory","2g")

spark=SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()
```

在之前的代码中，我们有上面这样一段代码。这里我们执行了一个 setMaster 的函数，这个函数其实就是在设置 Spark 的运行环境的(K8s)。接下来，让我们来了解下，Spark 的运行环境。

<div align="center"><img src="asset/1613724620336.jpg" /></div>

Spark 作为一个数据处理框架和计算引擎，被设计在所有常见的集群环境中运行, 在国内工作中主流的环境为 Yarn，不过逐渐容器式环境也慢慢流行起来。接下来，我们就分别看看不同环境下 Spark 的运行。

## Local 模式

想啥呢，你之前一直在使用的模式可不是 Local 模式哟。所谓的 Local 模式，就是不需要其他任何节点资源就可以在本地执行 Spark 代码的环境，一般用于教学，调试，演示等，我们之前使用的并不是这种。

## Standalone 模式

local 本地模式毕竟只是用来进行练习演示的，真实工作中还是要将应用提交到对应的集群中去执行，这里我们来看看只使用 Spark 自身节点运行的集群模式，也就是我们所谓的 独立部署(Standalone)模式。Spark 的 Standalone 模式体现了经典的 master-slave 模式。 集群规划:

<div align="center"><img src="asset/1613725147329.jpg" /></div>

这也是我们部署的方式，这种方式，当我们在部署的机器上运行 jps 时，会有以下的一些进程。

```
================linux1================
3330 Jps
3238 Worker
3163 Master
================linux2================
2966 Jps
2908 Worker
================linux3================
2978 Worker
3036 Jps
```

使用这种方式的时候，我们可以使用 Spark 自带的 UI 界面来管理 Job， UI 界面的地址为 http://<spark_host>:8080/.

## 配置高可用(HA)

所谓的高可用是因为当前集群中的 Master 节点只有一个，所以会存在单点故障问题。所以为了解决单点故障问题，需要在集群中配置多个 Master 节点，一旦处于活动状态的 Master 发生故障时，由备用 Master 提供服务，保证作业可以继续执行。这里的高可用一般采用 Zookeeper 设置, 集群规划：

<div align="center"><img src="asset/1613725521838.jpg" /></div>


## Yarn 模式

独立部署(Standalone)模式由 Spark 自身提供计算资源，无需其他框架提供资源。这种方式降低了和其他第三方资源框架的耦合性，独立性非常强。但是你也要记住，Spark 主 要是计算框架，而不是资源调度框架，所以本身提供的资源调度并不是它的强项，所以还是 和其他专业的资源调度框架集成会更靠谱一些。所以接下来我们来学习在强大的 Yarn 环境 下 Spark 是如何工作的(其实是因为在国内工作中，Yarn 使用的非常多)。

## K8S & Mesos 模式

Mesos 是 Apache 下的开源分布式资源管理框架，它被称为是分布式系统的内核,在 Twitter 得到广泛使用,管理着 Twitter 超过 30,0000 台服务器上的应用部署，但是在国内，依然使用着传统的 Hadoop 大数据框架，所以国内使用 Mesos 框架的并不多，但是原理其实都差不多，这里我们就不做过多讲解了。

<div align="center"><img src="asset/1613725831934.jpg" /></div>

容器化部署是目前业界很流行的一项技术，基于 Docker 镜像运行能够让用户更加方便 地对应用进行管理和运维。容器管理工具中最为流行的就是 Kubernetes(k8s)，而 Spark 也在最近的版本中支持了 k8s 部署模式。这里我们也不做过多的讲解。给个链接大家自己感受一下:https://spark.apache.org/docs/latest/running-on-kubernetes.html

<div align="center"><img src="asset/1613725936826.jpg" /></div>

## 部署模式对比

<div align="center"><img src="asset/1613726084985.jpg" /></div>

**使用到的一些端口号**

- Spark查看当前Spark-shell运行任务情况端口号:4040(计算)
- Spark Master 内部通信服务端口号:7077
- Standalone模式下，SparkMasterWeb端口号:8080(资源)
- Spark历史服务器端口号:18080
- HadoopYARN任务运行情况查看端口号:8088